In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
from langchain.utilities.wikipedia import WikipediaAPIWrapper
from langchain.document_loaders.web_base import WebBaseLoader
import yfinance
import json


def get_information_duckduckgo(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    search_for_anything_duck = inputs["search_for_anything_duck"]
    return ddg.run(f"User Search for {search_for_anything_duck}")


def get_information_wikipedia(inputs):
    wiki = WikipediaAPIWrapper()
    search_for_anything_wiki = inputs["search_for_anything_wiki"]
    return wiki.run(f"User Search for {search_for_anything_wiki}")


def get_text_from_url(inputs):
    loader = WebBaseLoader(inputs["url"])
    docs = loader.load()
    return docs


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_information_duckduckgo": get_information_duckduckgo,
    "get_information_wikipedia": get_information_wikipedia,
    "get_text_from_url": get_text_from_url,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_information_duckduckgo",
            "description": "The query you will search for. Example query: Research about the XZ backdoor",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_for_anything_duck": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["search_for_anything_duck"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_information_wikipedia",
            "description": "The query you will search for. Example query: Research about the XZ backdoor",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_for_anything_wiki": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["search_for_anything_wiki"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_text_from_url",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [41]:
import openai as client

# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
# assistant
assistant_id = "asst_clhG8awYQ8VSrYO86Cest8U7"

'asst_clhG8awYQ8VSrYO86Cest8U7'

In [4]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_MErcuOfRNR0Ig1JjuPqQFSNp', created_at=1713103007, metadata={}, object='thread')

In [5]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_dxQZO9YyXTZZ8fuFmHDRavE1', assistant_id='asst_clhG8awYQ8VSrYO86Cest8U7', cancelled_at=None, completed_at=None, created_at=1713103012, expires_at=1713103612, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_MErcuOfRNR0Ig1JjuPqQFSNp', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type':

In [37]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [38]:
get_tool_outputs(run.id, thread.id)

AttributeError: 'NoneType' object has no attribute 'submit_tool_outputs'

In [16]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "CRM"}
Calling function: get_balance_sheet with arg {"ticker": "CRM"}
Calling function: get_daily_stock_performance with arg {"ticker": "CRM"}


Run(id='run_dxQZO9YyXTZZ8fuFmHDRavE1', assistant_id='asst_clhG8awYQ8VSrYO86Cest8U7', cancelled_at=None, completed_at=None, created_at=1713103012, expires_at=1713103612, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713103027, status='queued', thread_id='thread_MErcuOfRNR0Ig1JjuPqQFSNp', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [33]:
get_run(run.id, thread.id).status

'completed'

In [34]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: The ticker symbol for Salesforce is CRM. To determine if Salesforce's stock is a good buy, we need to analyze its most recent financial statements, such as the income statement and balance sheet, as well as look at its stock performance over the last 100 days.

Let's proceed to obtain the necessary financial data and stock performance information for Salesforce (CRM).
assistant: The financial data and stock performance for Salesforce (CRM) have been retrieved:

**Income Statement Highlights:**
- Total Revenue for the most recent quarter (as of April 2023) was approximately $34.86 billion.
- Gross Profit was approximately $26.32 billion for the same period.
- Operating Income was around $5.999 billion.
- Net Income was approximately $4.136 billion.

**Balance Sheet Highlights:**
- Total Assets were $99.82 billion.
- Total Liabilities were $40.18 billion.
- Shareholders' Equity was approximately $59.65 billion.
- Total

In [35]:
send_message(thread.id, "Now I want to know if Cloudflare is a good buy.")

Message(id='msg_KnyMu2vHcwsC7Jagjt5JgzOz', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Now I want to know if Cloudflare is a good buy.'), type='text')], created_at=1713103076, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_MErcuOfRNR0Ig1JjuPqQFSNp')